Задание

Взять датасет
https://huggingface.co/datasets/merionum/ru_paraphraser
решить задачу парафраза

(дополнительно необязательная задача)на выбор взять
https://huggingface.co/datasets/sberquad
https://huggingface.co/datasets/blinoff/medical_qa_ru_data
натренировать любую модель для вопросно ответной системы
как альтернатива можно взять любой NER датасет из https://github.com/natasha/corus#reference и обучить NER

### Решаем задачу парафраза

In [1]:
# Загрузка датасета

from datasets import load_dataset

dataset = load_dataset("merionum/ru_paraphraser")

dataset

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /Users/aleksandrkrylov/.cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-ec474e58a96b8289/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 7227
    })
    test: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 1924
    })
})

In [2]:
# Загрузка предобученных моделей и пайплайна

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rut5-base-paraphraser")

model = AutoModelForSeq2SeqLM.from_pretrained("cointegrated/rut5-base-paraphraser")

/Users/aleksandrkrylov/anaconda3/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [3]:
# Создадим функцию для парафраза

def paraphrase(text, beams=6, grams=3, do_sample=False):
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    max_size = int(x.input_ids.shape[1] * 1.5 + 10)
    out = model.generate(**x, encoder_no_repeat_ngram_size=grams, num_beams=beams, max_length=max_size, do_sample=do_sample)
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [5]:
# Примеры работы парафраза

for i in range(10):
    print('Initial text: ', dataset["train"][i]['text_1'])
    print('Parafrase: ', paraphrase(dataset["train"][i]['text_1']))
    print('------------------------------------------------------------')

Initial text:  Полицейским разрешат стрелять на поражение по гражданам с травматикой.
Parafrase:  Полиция разрешит стрелять в поражение гражданам, имеющим травматизм.
------------------------------------------------------------
Initial text:  Право полицейских на проникновение в жилище решили ограничить.
Parafrase:  Решено ограничить право полиции проникнуть в жилье.
------------------------------------------------------------
Initial text:  Президент Египта ввел чрезвычайное положение в мятежных городах.
Parafrase:  В Египте президент ввел особое положение по мятежным городам.
------------------------------------------------------------
Initial text:  Вернувшихся из Сирии россиян волнует вопрос трудоустройства на родине.
Parafrase:  Вопрос трудоустройства россиян, вернувшихся из Сирии, волнует возвращающихся из страны.
------------------------------------------------------------
Initial text:  В Москву из Сирии вернулись 2 самолета МЧС с россиянами на борту.
Parafrase:  На борту Москв